In [ ]:
import pandas as pd
import numpy as np
import io
import re
import csv
from spacy.lang.id import Indonesian
from itertools import zip_longest
nlp_id_spacy = Indonesian()
pos_term = []
pos_term_neg = []
csv.register_dialect('myDialect', delimiter=',')

In [ ]:
# lokasi untuk file barasa sebagai sumber lexical pada tugas akhir
df = pd.read_csv('barasa.csv', delimiter='\t', index_col=False)

# file_input sebagai variabel sumber file tweet yang telah didapat dari proses preprocessing
file_input = "data_ALL.txt"
file = open(file_input, mode='r', encoding='utf-8')
text = file.readlines()

# variabel file untuk menyimpan hasil proses sentiwordnet. Ganti nama file jika dibutuhkan
file = "data_DIC_TOTAL.csv"
senti_dic = open(file, mode='w')

In [ ]:
# fungsi untuk memanggil kamus kelas kata (POS Tagging Bahasa Indonesia). ganti file sesuai dengan lokasi kamus kata
def loadPos_id(file = 'data/kata_dasar.txt'):
    kata_pos = {}
    df=open(file,"r",encoding="utf-8", errors='replace')
    data=df.readlines();df.close()
    
    for line in data:
        d = line.split()
        kata = d[0].strip()
        
        pos = d[-1].strip().replace("(",'').replace(')','')
        kata_pos[kata] = pos
        
    return kata_pos

In [ ]:
print(df[df['lemma'].isnull()])

In [ ]:
# fungsi untuk memberikan kelas kata pada setiap kata tweet yang telah didapat
kata_pos = loadPos_id()
pos_term.clear()

for idx, texts in enumerate(text):
    tweet = texts.split(",")
    label = tweet[1].split("\n")
    wordList = tweet[0].split()
    
    for term in wordList:
        try:
            pos_term.append([idx, term , kata_pos[term], label[0]]) 
        except:
            pos_term.append([idx, term , 'n', label[0]]) 

In [ ]:
pos_term

In [ ]:
%%time
d = []
q = []
tweet_id = 'tweet_id'
term = 'term'
pos = 'pos'
pos_fre = 'pos_fre'
neg_fre = 'neg_fre'
label = 'label'
q.append(tweet_id)
q.append(term)
q.append(pos)
q.append(pos_fre)
q.append(neg_fre)
q.append(label)

# file csv untuk menyimpan hasil sentiwordnet 
csv_save = csv.writer(senti_dic,dialect='myDialect')
csv_save.writerow(q)

for itter in range(len(pos_term)):
    count_positif = 0
    count_negatif = 0

    item = df[df['lemma']==pos_term[itter][1]]
    if pos_term[itter][2] == "adj":
        pos_synset = "a"
    elif pos_term[itter][2] == "n":
        pos_synset = "n"
    elif pos_term[itter][2] == "v":
        pos_synset = "v"
    elif pos_term[itter][2] == "k":
        pos_synset = "r"
        
    item = item[item['language'].str.contains("I|B")]
    item = item[item['synset'].str.contains(pos_synset)]
    item = item[item['goodness'].str.contains("Y|O|M")]
    item = item[item['PosScore']-item['NegScore'] != 0]
    
    item_POS = item[item['PosScore']-item['NegScore'] > 0]
    item_NEG = item[item['PosScore']-item['NegScore'] < 0]

    if len(item.index) > 0:
        if len(item_POS) > 0:
            count_positif += len(item_POS)   
        if len(item_NEG) > 0:
            count_negatif += len(item_NEG)
        
        # save csv
        d.append(pos_term[itter][0])
        d.append(pos_term[itter][1])
        d.append(pos_term[itter][2])
        d.append(count_positif)
        d.append(count_negatif)
        d.append(pos_term[itter][3])
        
        csv_save = csv.writer(senti_dic,dialect='myDialect')
        csv_save.writerow(d)
        
    d.clear()   
    
senti_dic.close()